# Ethereum Consensus Mechanisms & Proof‑of‑Stake (Notebook 1)
**Sources**: ethereum.org consensus & PoS docs (see in‑app message for links).

---
## What is a consensus mechanism?
- Full stack of protocols + incentives for nodes to agree on state.
- Ethereum uses a **PoS-based** mechanism with rewards/penalties and fork choice.

### Time structure
- **Slot** = 12 seconds
- **Epoch** = 32 slots

### Roles
- **Proposer**: proposes a block for the slot
- **Attesters (committee)**: vote on source/target/head

### Finality (Casper FFG)
- Checkpoints at epoch boundaries; if 2/3 stake links two checkpoints -> the earlier becomes *finalized*.

In [ ]:
# Parameters for time
SLOT_SECONDS = 12
SLOTS_PER_EPOCH = 32

print(f"Slot = {SLOT_SECONDS}s, Epoch = {SLOTS_PER_EPOCH} slots = {SLOT_SECONDS*SLOTS_PER_EPOCH/60:.2f} minutes")

## Toy validator set & committees
We'll simulate a small validator set and random committee assignments per slot. This is **not** the real RANDAO / shuffling, just a teaching demo.

In [ ]:
import random, math
random.seed(42)

N_VALIDATORS = 256  # tiny vs mainnet
VALIDATORS = list(range(N_VALIDATORS))

def committee_for_slot(slot, size=64):
    # very simplified: deterministic pseudo-random using slot as seed
    rng = random.Random(slot)
    members = VALIDATORS.copy()
    rng.shuffle(members)
    return members[:size]

# Show a few committees
for s in range(3):
    c = committee_for_slot(s, size=8)
    print(f"Slot {s}: committee -> {c}")

## LMD‑GHOST‑style toy fork choice
In Ethereum PoS, the fork choice is **LMD‑GHOST**: pick the head with greatest weight of latest messages. Below is a **toy** scoring to illustrate the concept.

In [ ]:
from collections import defaultdict, namedtuple

Block = namedtuple('Block', 'hash parent slot')
# Build a tiny fork tree
genesis = Block('G', None, -1)
b1 = Block('A', 'G', 0)
b2 = Block('B', 'A', 1)  # main
b3 = Block('C', 'A', 1)  # competing
b4 = Block('D', 'B', 2)  # extends B
tree = {blk.hash: blk for blk in [genesis,b1,b2,b3,b4]}

# Latest attestation per validator: map validator->block_hash they voted head for
latest = {}
committee = committee_for_slot(2, size=20)
# Pretend most attesters saw 'D', a few saw 'C'
for i,v in enumerate(committee):
    latest[v] = 'D' if i < 17 else 'C'

def children(parent_hash):
    return [h for h,b in tree.items() if b.parent == parent_hash]

def weight_on(block_hash):
    # weight = number of latest messages whose chain contains block_hash
    def on_chain(h, target):
        while h and h != 'G':
            if h == target: 
                return True
            h = tree[h].parent
        return target=='G'
    return sum(1 for v,h in latest.items() if on_chain(h, block_hash))

def ghost_head(start='G'):
    head = start
    while True:
        cs = children(head)
        if not cs: 
            return head
        # pick child with max weight; break ties lexicographically for determinism
        head = sorted(cs, key=lambda x: (-weight_on(x), x))[0]

print("GHOST head =", ghost_head())

> In this toy, the head should select the branch with the greater **attestation weight**.

## End‑to‑end: transaction inclusion walkthrough (didactic)
1. User sends tx to **execution client** mempool.
2. Slot proposer builds execution payload, wraps into **beacon block**.
3. Peers re‑execute payload; committee **attests**.
4. After supermajority links between checkpoints, the earlier checkpoint **finalizes**.